# Databricks Cost to Slack 🧱

![](https://img.shields.io/badge/Slack-4A154B.svg?style=for-the-badge&logo=Slack&logoColor=white)
![](https://img.shields.io/badge/Databricks-FF3621.svg?style=for-the-badge&logo=Databricks&logoColor=white)

> Authors: [Eshwaran Venkat](github.com/cricksmaidiene)

In [0]:
!pip install tabulate -q

In [0]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import os
import requests

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("DAY_COUNT", "2")
dbutils.widgets.text("SLACK_WEBHOOK", "")

N: int = int(dbutils.widgets.get("DAY_COUNT"))  # Example value for N days ago, adjust as needed
SLACK_WEBHOOK = dbutils.widgets.get("SLACK_WEBHOOK")

assert all([N, SLACK_WEBHOOK]), "One or more required parameters not set"

In [0]:
# The message payload using Block Kit for formatting
message_payload = {
    "blocks": [
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": ""
            }
        },
        {
            "type": "divider"
        },
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": ""
            }
        }
    ]
}

In [0]:
df = spark.sql(
    f"""
    SELECT *, identity_metadata.run_as as user FROM system.billing.usage 
    WHERE usage_date >= date_trunc('day', NOW()) - interval '{N} day' 
    AND usage_date < date_trunc('day', NOW()) - interval '{N-1} day'
    """
).toPandas()

print(df.shape)
df.head()

In [0]:
dbu_usd_prices = spark.sql(
    f"""
    SELECT sku_name, round(pricing.default, 2) as usd_price 
    FROM system.billing.list_prices 
    WHERE sku_name in (
        SELECT sku_name FROM system.billing.usage 
        WHERE usage_date >= date_trunc('day', NOW()) - interval '{N} day' 
    AND usage_date < date_trunc('day', NOW()) - interval '{N-1} day'
    )
    """
).toPandas()

print(dbu_usd_prices.shape)
dbu_usd_prices.head()

In [0]:
df["usd"] = (
    df["sku_name"].map(dbu_usd_prices.set_index("sku_name").to_dict()["usd_price"])
    * df["usage_quantity"]
)
df["usd"] = df["usd"].astype(float).round(2)
print(df["usd"].info())

In [0]:
report = df.groupby(["billing_origin_product"])[["usd"]].sum().reset_index()
report.columns = ["product", "cost"]
report['product'] = report['product'].str.replace("_", " ").str.title()
report

In [0]:
usage_date = df["usage_date"].iloc[0].strftime(r"%b %d")
day_cost = report["cost"].sum()
message_color = "info" if day_cost < 22 else "error"

message_title = f"🧱 *Databricks Cost* for *{usage_date}* is *${day_cost:,.2f}*"
message_log = f"```{report.to_markdown(index=False)}```"

message_payload['blocks'][0]['text']['text'] = message_title
message_payload['blocks'][2]['text']['text'] = message_log

In [0]:
# Sending the POST request to the Slack webhook URL
response = requests.post(SLACK_WEBHOOK, json=message_payload)
assert response.status_code == 200, "Unable to post to slack"